# LA's COVID-19 Reopening / Recovery Indicators 

The NYT wrote about rushed reopenings being [a global trial-and-error experiment](https://www.nytimes.com/2020/05/07/world/europe/coronavirus-reopening-costs.html), that most US states reopening [fail to meet the WH guidelines](https://www.nytimes.com/interactive/2020/05/07/us/coronavirus-states-reopen-criteria.html), and Vox has written about [the importance of COVID testing indicators](https://www.vox.com/2020/5/8/21249880/coronavirus-testing-covid-reopening-economy-lockdowns-social-distancing)

[NY State's Reopening Indicators](https://www.nytimes.com/2020/05/04/nyregion/coronavirus-reopen-cuomo-ny.html) were based heavily in part by the White House [Opening Up America Again gating criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf).

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt

## [Symptoms](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf)
* Downward trajectory of influenza-like illnesses (ILI) reported within a 14-day period **and**
* Downward trajectory of COVID-like syndromic cases reported within a 14-day period

## Cases
* Downward trajectory of documented cases within a 14-day period **or**
* Downward trajectory of positive tests as a percent of total tests within a 14-day period (flat or increasing volume of tests) 

### [NY State Indicators](https://www.governor.ny.gov/news/amid-ongoing-covid-19-pandemic-governor-cuomo-outlines-additional-guidelines-when-regions-can)
* At least 14 days decline in total net hospitalizations on a 3-day rolling average **or** less than 15 net new total hospitalizations on a 3-day rolling average **and**
* At least 14 days decline in total net deaths on a 3-day rolling average **or** less than 5 new deaths on a 3-day rolling average

### [NY Times Indicators](https://www.nytimes.com/interactive/2020/05/07/us/coronavirus-states-reopen-criteria.html)
* At least 14 days decline in new cases on a 7-day rolling average

## Hospitals
* Treat all patients without crisis care **and**
* Robust testing program in place for at-risk healthcare workers, including emerging antibody testing

### NY State Indicators
* At least 30% total hospital and ICU beds available **and**
* At least 90 days of PPE stockpiled **and**
* Capacity to conduct 30 tests per 1,000 residents per month **and**
* 30 working contact tracers per 100,000 residents

For regions with fewer COVID cases:
* Fewer than 2 new COVID patients admitted per 100,000 residents per day

In [52]:
US_COUNTY_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/8aba663239fc428f8bcdc48e213e3172/data"
full_df = pd.read_csv(US_COUNTY_URL, dtype={'fips':'str'})

def case_indicators_county(df, county_name, start_date): 
    US_COUNTY_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/8aba663239fc428f8bcdc48e213e3172/data"
    full_df = pd.read_csv(US_COUNTY_URL, dtype={'fips':'str'})
    
    keep_cols = ['county', 'state', 'fips', 'date', 'Lat', 'Lon', 'cases',
           'deaths', 'new_cases', 'new_deaths']

    full_df['date'] = pd.to_datetime(full_df.date)
    
    df = (full_df[(full_df.county==county_name) & (full_df.date >=start_date)][keep_cols]
          .sort_values(['county', 'state', 'fips', 'date'])
          .reset_index(drop=True)
    )

    # Derive new columns
    df = df.assign(
        # 7-day rolling average for new cases
        cases_avg7 = df.new_cases.rolling(window=7).mean(),
        # 3-day rolling average for new deaths
        deaths_avg3 = df.new_deaths.rolling(window=3).mean(),
    )
    
    # Make cases charts
    cases_chart = alt.Chart(df).mark_line().encode(
        x='date',
        y=alt.Y('cases_avg7', 
                title='7-day avg',
               ),
        color = alt.value('navy')
    ).properties(
        title=f'{county_name} County: New Cases',
    ).configure_title(
        fontSize=16,
        font='Roboto',
        anchor='middle',
        color='Black'
    ).configure_axis(
        gridOpacity = 0.4
    )

    # Make deaths chart
    deaths_chart = alt.Chart(df).mark_line().encode(
        x='date',
        y=alt.Y('deaths_avg3', 
                title='3-day avg',
               ),
        color = alt.value('maroon')
    ).properties(
        title=f'{county_name} County: Deaths',
    ).configure_title(
        fontSize=16,
        font='Roboto',
        anchor='middle',
        color='Black'
    ).configure_axis(
        gridOpacity = 0.4
    )
    
    display(cases_chart)
    display(deaths_chart)
    
    return df

In [53]:
la = case_indicators_county(full_df, "Los Angeles", "3/1/20")

alt.Chart(...)

alt.Chart(...)

In [ ]:
def state_indicators(df, state_name):
    df = pd.read_csv(US_COUNTY_URL)

    keep_cols = ['date', 'state_cases', 'state_deaths', 'new_state_cases', 'new_state_deaths']
    return df